<a href="https://colab.research.google.com/github/JennyFrost/trial_task/blob/main/trial_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy --no-cache-dir
!python -m spacy download en_core_web_lg
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 129.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2024-01-20 23:03:37.603881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 23:03:37.603953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: 

In [44]:
import numpy as np
import re
from itertools import chain
import torch
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read the input text and the standardised phrases ##

In [3]:
filename = input('Enter the path to the file with the text you wish to improve: ')

# place the text file in the folder 'content'
if filename:
    with open(file=filename, mode='r') as f:
        lines = f.readlines()
    input_text = ' '.join([line.strip() for line in lines if not re.match(r'\s+', line)])
    input_text

Enter the path to the file with the text you wish to improve: /content/text.txt


In [5]:
# place the file with standard phrases in the folder 'content'
filename_phrases = '/content/business phrases.txt'
with open(file=filename_phrases, mode='r') as f:
    lines = f.readlines()
standard_phrases = list(set([phrase.strip().lower() for phrase in lines]))
len(standard_phrases)

526

## Extracting verb and noun phrases from text ##

In [4]:
def print_sentence_decomposition(sentence_doc, print_sentence=True, print_lefts_and_rights=True):
    if print_sentence:
        print(sentence_doc, "\n")

    if print_lefts_and_rights:
        for token in sentence_doc:
            print("{} {:25s} {:10s} {:10s} {:20s} {:20s} {}".format(token.i, token.text, token.pos_, token.dep_,
                                                                    token.head.text, "['" + "','".join(
                    [x.text for x in token.lefts]) + "']", "['" + "','".join([x.text for x in token.rights]) + "']"))

    else:
        for token in sentence_doc:
            print("{} {:25s} {:10s} {:10s} {:20s}".format(token.i, token.text, token.pos_, token.dep_, token.head.text))

In [48]:
class PhraseExtractor:

    def __init__(self, docs: list[spacy.tokens.doc.Doc]):
        self.docs = docs

    @staticmethod
    def get_action_verbs(doc: spacy.tokens.doc.Doc) -> list[spacy.tokens.token.Token]:
        """
            For a sentence processed with spaCy, finds all the verbs

            Input
            -----
            doc : Doc
                sentence processed with spaCy
            Output
            ------
            verbs: list[Token]
                list of the verb tokens found (in spaCy format)

        """
        verbs = []
        for token in doc:
            if token.pos_ in ('VERB', 'AUX'):
                if token.dep_ == 'amod' and token.head.dep_ != 'ROOT':
                    continue
                if token.i >= 2:
                    if (re.search(r'ed\b', token.text)
                                            and (((doc[token.i-1].text == ','
                                            and doc[token.i-2].pos_ == 'ADJ')
                                            or doc[token.i-1].pos_ == 'ADJ')
                                            or (token.dep_ in ('conj','appos')
                                            and not re.search(r'ed\b', token.head.text)))):
                        continue
                verbs.append(token)
            elif token.pos_ == 'ADJ' and bool(re.search(r'ed\b|ing\b', token.text)) \
                    and token.head.dep_ in ('ROOT', 'nsubj') \
                    and not ((doc[token.i - 1].text == ',' and
                              doc[token.i - 2].pos_ == 'ADJ') or
                              doc[token.i - 1].pos_ == 'ADJ'):
                verbs.append(token)
        return verbs

    @staticmethod
    def get_verb_objects(doc: spacy.tokens.doc.Doc,
                         verb: spacy.tokens.token.Token) -> list[str]:
        """
            For sentence processed with spacy and verb processed with spacy finds direct objects
            and prepositional phrases of the verb

            Input
            -----
            doc : Doc
                sentence processed with spacy
            verb: Doc
                verb processed with spacy
            Output
            ------
            obj_text/pobj_text: list
                a list of strings (objects of the verb) with left children
        """
        if 'dobj' in list(map(lambda x: x.dep_, verb.rights)):
            obj = [tok for tok in verb.rights if tok.dep_ == 'dobj'][0]
            rights = list(chain([x for x in verb.rights if x.i > obj.i], list(obj.rights)))
            objs = [obj]
            obj_text = [' '.join([x.text for x in verb.rights if x.i < obj.i])]
            if obj.conjuncts:
                conjs = [doc[conj.i:conj.i+1] for conj in obj.conjuncts]
                rights += list(chain.from_iterable([list(conj[-1].rights) for conj in conjs]))
                objs += conjs
                obj_text += [conj.text for conj in conjs]
            for i, obj in enumerate(objs):
                lefts = [tok.text for tok in obj.subtree if tok.i <= obj.i]
                if lefts:
                    obj_text[i] += ' '.join(lefts)
            if 'ADP' in list(map(lambda x: x.pos_, rights)):
                prep = list(filter(lambda x: x.pos_ == 'ADP', rights))[0]
                for i, obj in enumerate(objs):
                    obj_text[i] += ' ' + prep.text
                if [tok for tok in prep.rights if tok.dep_ == 'pobj']:
                    pobj = [tok for tok in prep.rights if tok.dep_ == 'pobj'][0]
                    pobj_lefts = [tok.text for tok in pobj.subtree if tok.i <= pobj.i or tok.pos_ == 'ADP']
                    if len(pobj_lefts) > 1:
                        for i, obj in enumerate(objs):
                            obj_text[i] += ' ' + ' '.join(pobj_lefts)
            return obj_text
        if any(list(map(lambda word: word.pos_ == 'ADP', verb.rights))):
            prep = [word for word in verb.rights if word.pos_ == 'ADP'][0]
            if [word for word in prep.rights if word.dep_ == 'pobj']:
                pobj = [word for word in prep.rights if word.dep_ == 'pobj'][0]
                pobjs = [pobj]
                pobj_text = [' '.join([x.text for x in verb.rights if x.i < pobj.i])]
                if pobj.conjuncts:
                    pobjs += list(pobj.conjuncts)
                    pobj_text.append(' '.join([conj.text for conj in pobj.conjuncts]))
                for i, obj in enumerate(pobjs):
                    lefts = [tok.text for tok in pobj.subtree if tok.i <= pobj.i]
                    if lefts:
                        pobj_text[i] += ' ' + ' '.join(lefts)
                    if 'ADP' in list(map(lambda x: x.pos_, pobj.rights)):
                        prep2 = list(filter(lambda x: x.pos_ == 'ADP', pobj.rights))[0]
                        if [tok for tok in prep2.rights if tok.dep_ == 'pobj']:
                            pobj2 = [tok for tok in prep2.rights if tok.dep_ == 'pobj'][0]
                            pobj_lefts2 = [tok.text for tok in pobj2.subtree if tok.i <= pobj2.i or tok.pos_ == 'ADP']
                            if len(pobj_lefts2) > 1:
                                for i, pobj in enumerate(pobjs):
                                    pobj_text[i] += ' ' + prep2.text + ' ' + ' '.join(pobj_lefts2)
                return pobj_text

    @staticmethod
    def get_noun_phrases(doc: spacy.tokens.doc.Doc) -> list[str]:
        """
            For sentence processed with spacy finds noun phrases (noun and its children)

            Input
            -----
            doc : Doc
                sentence processed with spacy
            Output
            ------
            noun_phrases: list[str]
                a list of strings of noun phrases
        """
        noun_phrases = []
        for token in doc:
            if token.pos_ in ('NOUN', 'PROPN'):
                conj = [tok for tok in token.subtree if tok.dep_ == 'conj']
                conj_ind = len(doc)
                if conj:
                    conj_ind = conj[0].i
                    if doc[conj_ind-1].pos_ in ('CCONJ', 'PUNCT'):
                        conj_ind -= 1
                else:
                    if [tok for tok in token.subtree if tok.pos_ == 'PUNCT']:
                        conj_ind = [tok for tok in token.subtree if tok.pos_ == 'PUNCT'][0].i
                noun_phrase = [tok for tok in token.subtree if tok.pos_ != 'PRON' and tok.i < conj_ind]
                if noun_phrase:
                    if noun_phrase[0].pos_ == 'DET':
                        noun_phrase = noun_phrase[1:]
                    if len(noun_phrase) > 1:
                        noun_phrases.append(' '.join(list(map(lambda x: x.text, noun_phrase))))
        return noun_phrases

    def get_phrases_from_text(self) -> tuple[list[str], list[str]]:
        """
            For a list of sentences processed with spacy, makes verb phrases of verb + its object
            and collects all the verb phrases and noun phrases

            Input
            -----
            doc : Doc
                sentence processed with spacy
            Output
            ------
            noun_phrases: list[str]
                a list of strings of noun phrases
            verb_phrases: list[str]
                a list of strings of verb phrases
        """
        verb_phrases_from_text = []
        noun_phrases_from_text = []
        for doc in self.docs:
            verbs = self.get_action_verbs(doc)
            print(doc)
            if verbs:
                for verb in verbs:
                    if self.get_verb_objects(doc, verb):
                        phrase = verb.lemma_ + ' ' + self.get_verb_objects(doc, verb)[0]
                        verb_phrases_from_text.append(phrase)
                        print(phrase)
            noun_phrases = self.get_noun_phrases(doc)
            if noun_phrases:
                noun_phrases_from_text.extend(noun_phrases)
                for phrase in noun_phrases:
                    print(phrase)
            print('=============================================================')
        return verb_phrases_from_text, noun_phrases_from_text

In [14]:
input_docs = list(nlp.pipe(sent_tokenize(input_text)))
input_docs

[I am thrilled to share some exciting news with you!,
 Our recent sales figures have shown a significant increase, and it's all thanks to the hard work and dedication of our team.,
 This is a fantastic achievement, and I wanted to take a moment to express my gratitude for your substantial contribution to this success.,
 Your efforts in driving sales and engaging with our customers have been instrumental in reaching our targets.,
 Additionally, I wanted to touch base regarding our ongoing marketing campaign.,
 The initial feedback and results have been quite promising.,
 The innovative approach and creative strategies employed by the marketing team are resonating well with our target audience, which in turn is positively impacting our sales.,
 As we continue to ride this wave of success, I believe it's crucial to maintain our momentum.,
 This would be a good time to review our sales tactics and align them even more closely with the marketing strategies to maximize our impact in the mark

In [50]:
standard_phrases_docs = list(nlp.pipe(standard_phrases))
standard_phrases_docs[:20]

[market potential,
 manage workflows,
 increase revenue,
 employee engagement,
 capital investment,
 globalization strategy,
 sales forecasting,
 lean management,
 business success,
 brand strategy,
 enterprise resource planning,
 decision making,
 strengthen capabilities,
 profit margins,
 execute plans,
 maximize output,
 negotiate deals,
 strategic objectives,
 operational planning,
 maximize potential]

In [51]:
verb_standard_phrases, noun_standard_phrases = [], []
for doc in standard_phrases_docs:
    root = [tok for tok in doc if tok.dep_ == 'ROOT'][0]
    if root.pos_ in ('VERB', 'AUX'):
        verb_standard_phrases.append(doc.text)
    else:
        noun_standard_phrases.append(doc.text)

print('Examples of verb phrases: \n', *verb_standard_phrases[:10], sep='\n', end='\n\n')
print('Examples of noun phrases: \n', *noun_standard_phrases[:10], sep='\n')

Examples of verb phrases: 

manage workflows
increase revenue
strengthen capabilities
execute plans
maximize output
negotiate deals
maximize potential
reduce waste
develop techniques
protect assets

Examples of noun phrases: 

market potential
employee engagement
capital investment
globalization strategy
sales forecasting
lean management
business success
brand strategy
enterprise resource planning
decision making


In [52]:
phrase_extractor = PhraseExtractor(input_docs)
verb_phrases_from_text, noun_phrases_from_text = phrase_extractor.get_phrases_from_text()

I am thrilled to share some exciting news with you!
share some exciting news with
exciting news
Our recent sales figures have shown a significant increase, and it's all thanks to the hard work and dedication of our team.
show a significant increase
recent sales figures
significant increase
thanks to the hard work
hard work
This is a fantastic achievement, and I wanted to take a moment to express my gratitude for your substantial contribution to this success.
take a moment
express my gratitude for your substantial contribution to
fantastic achievement
gratitude for substantial contribution to this success
substantial contribution to this success
Your efforts in driving sales and engaging with our customers have been instrumental in reaching our targets.
drive sales
engage with our customers
reach our targets
efforts in driving sales
Additionally, I wanted to touch base regarding our ongoing marketing campaign.
touch base
ongoing marketing campaign
The initial feedback and results have b

## Sentence embeddings from Sentence Transformers ##

In [54]:
model1 = SentenceTransformer('all-distilroberta-v1')

In [55]:
verb_phrases_from_text_embeddings = model1.encode(verb_phrases_from_text, convert_to_tensor=True)
noun_phrases_from_text_embeddings = model1.encode(noun_phrases_from_text, convert_to_tensor=True)
verb_standard_phrases_embeddings = model1.encode(verb_standard_phrases, convert_to_tensor=True)
noun_standard_phrases_embeddings = model1.encode(noun_standard_phrases, convert_to_tensor=True)

In [24]:
verb_cosine_scores = util.cos_sim(verb_phrases_from_text_embeddings, verb_standard_phrases_embeddings).numpy()
noun_cosine_scores = util.cos_sim(noun_phrases_from_text_embeddings, noun_standard_phrases_embeddings).numpy()

In [26]:
# verb phrases and replacements
for i, phrase in enumerate(verb_phrases_from_text):
    j = np.argmax(verb_cosine_scores[i])
    if verb_cosine_scores[i][j] > 0.5:
        print("{:50s} {:30s} Score: {:.4f}".format(phrase, verb_standard_phrases[j], verb_cosine_scores[i][j]))

show a significant increase                        sales increase                 Score: 0.5712
drive sales                                        drive sales                    Score: 1.0000
engage with our customers                          enhance customer experience    Score: 0.7009
reach our targets                                  achieve targets                Score: 0.6980
employ by the marketing team                       target customers               Score: 0.5131
resonate well with our target audience             target demographics            Score: 0.5010
impact our sales                                   increase sales                 Score: 0.7293
ride this wave of success                          build success                  Score: 0.5183
review our sales tactics                           target customers               Score: 0.5464
align them with the marketing strategies in        manage strategies              Score: 0.5593
maximize our impact in the market       

In [28]:
# noun phrases and replacements
for i, phrase in enumerate(noun_phrases_from_text):
    j = np.argmax(noun_cosine_scores[i])
    if noun_cosine_scores[i][j] > 0.5:
        print("{:50s} {:50s} Score: {:.4f}".format(phrase, noun_standard_phrases[j], noun_cosine_scores[i][j]))

significant increase                               revenue growth                                     Score: 0.5189
substantial contribution to this success           business success                                   Score: 0.5035
efforts in driving sales                           sales objectives                                   Score: 0.6221
ongoing marketing campaign                         marketing campaign                                 Score: 0.8422
innovative approach and creative                   business innovation                                Score: 0.6068
marketing team                                     marketing campaign                                 Score: 0.6900
wave of success                                    business success                                   Score: 0.5701
good time to review sales tactics                  sales strategy                                     Score: 0.5932
sales tactics                                      sales techniques     

## Word embeddings from RoBERTa ##

In [29]:
def mean_pooling(model_output, attention_mask):
    """
        Obtains word embeddings from the model and averages them to get a sentence embedding
    """
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [56]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model2 = RobertaModel.from_pretrained("roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model2.to(device)
model2.eval()

verb_phrases_from_text_emb = torch.Tensor().to(device)
verb_standard_phrases_emb = torch.Tensor().to(device)
noun_phrases_from_text_emb = torch.Tensor().to(device)
noun_standard_phrases_emb = torch.Tensor().to(device)

with torch.no_grad():
    for vp in verb_phrases_from_text:
        inputs = tokenizer(vp, return_tensors="pt")
        outputs = model2(**inputs)
        verb_phrases_from_text_emb = torch.cat([verb_phrases_from_text_emb,
                                                mean_pooling(outputs, inputs['attention_mask'])])
    verb_phrases_from_text_emb = verb_phrases_from_text_emb.cpu().numpy()
    for vp in verb_standard_phrases:
        inputs = tokenizer(vp, return_tensors="pt")
        outputs = model2(**inputs)
        verb_standard_phrases_emb = torch.cat([verb_standard_phrases_emb,
                                               mean_pooling(outputs, inputs['attention_mask'])])
    verb_standard_phrases_emb = verb_standard_phrases_emb.cpu().numpy()
    for np in noun_phrases_from_text:
        inputs = tokenizer(np, return_tensors="pt")
        outputs = model2(**inputs)
        noun_phrases_from_text_emb = torch.cat([noun_phrases_from_text_emb,
                                                mean_pooling(outputs, inputs['attention_mask'])])
    noun_phrases_from_text_emb = noun_phrases_from_text_emb.cpu().numpy()
    for np in noun_standard_phrases:
        inputs = tokenizer(np, return_tensors="pt")
        outputs = model2(**inputs)
        noun_standard_phrases_emb = torch.cat([noun_standard_phrases_emb,
                                               mean_pooling(outputs, inputs['attention_mask'])])
    noun_standard_phrases_emb = noun_standard_phrases_emb.cpu().numpy()

In [58]:
verb_cosine_scores = cosine_similarity(verb_phrases_from_text_emb, verb_standard_phrases_emb)
noun_cosine_scores = cosine_similarity(noun_phrases_from_text_emb, noun_standard_phrases_emb)

In [59]:
verb_cosine_scores

array([[0.94674265, 0.9535094 , 0.93963057, ..., 0.9481195 , 0.94369954,
        0.9481355 ],
       [0.946267  , 0.9693363 , 0.94950885, ..., 0.94981474, 0.94590247,
        0.9585757 ],
       [0.9203085 , 0.9344268 , 0.9246465 , ..., 0.9277223 , 0.92426074,
        0.93582684],
       ...,
       [0.95567465, 0.9663379 , 0.9492316 , ..., 0.9536645 , 0.9518582 ,
        0.9675814 ],
       [0.95434606, 0.9592153 , 0.941956  , ..., 0.9518529 , 0.9450792 ,
        0.9603782 ],
       [0.96144027, 0.964925  , 0.9429299 , ..., 0.9551538 , 0.94812214,
        0.9633505 ]], dtype=float32)

In [60]:
for i, phrase in enumerate(verb_phrases_from_text):
    j = np.argmax(verb_cosine_scores[i])
    if verb_cosine_scores[i][j] > 0.5:
        print("{:50s} {:30s} Score: {:.4f}".format(phrase, verb_standard_phrases[j], verb_cosine_scores[i][j]))

AttributeError: 'str' object has no attribute 'argmax'

In [ ]:
for i, phrase in enumerate(noun_phrases_from_text):
    j = np.argmax(noun_cosine_scores[i])
    if noun_cosine_scores[i][j] > 0.5:
        print("{:50s} {:50s} Score: {:.4f}".format(phrase, noun_standard_phrases[j], noun_cosine_scores[i][j]))